In [1]:
# Load the necessary package
library(readr)

# Specify the file path
file_path <- "../data/us_ind_by_month.csv"

# Read the CSV file
data <- read_csv(file_path)

# Load the necessary packages
library(ggplot2)

New names:
• `` -> `...1`
Rows: 6048 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): Month, NAICS 2022 3-Digit Industry Group (Name)
dbl (7): ...1, Year, Year-Month, NAICS 2022 3-Digit Industry Group, Percent,...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [2]:
# we group the data by year and 
# first multiply N by Percent to arrive at the number of WFH jobs
# second we add up the number of WFH jobs for each year
# third we devide the number of WFH jobs for each year by the total
# number of jobs for that year to get the percentage of WFH jobs

In [3]:
library(dplyr)

# Assuming your data frame is named 'data' and it has columns 'Year', 'N', and 'Percent' # nolint
data_by_year_NAICS <- data %>% # nolint
  group_by(`Year`,`NAICS 2022 3-Digit Industry Group`,`NAICS 2022 3-Digit Industry Group (Name)`) %>% # nolint
  mutate(WFH_jobs = N * Percent / 100) %>%
  summarise(Total_WFH_jobs = sum(WFH_jobs),
            Total_jobs = sum(N),
            Percent_WFH_jobs = Total_WFH_jobs / Total_jobs * 100)


Attaching package: ‘dplyr’




The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


`summarise()` has grouped output by 'Year', 'NAICS 2022 3-Digit Industry
Group'. You can override using the `.groups` argument.


In [4]:
# keep only data from 2019
data_by_year_NAICS_2019 <- data_by_year_NAICS %>% filter(Year == 2019)
# sort by Percent_WFH_jobs
data_by_year_NAICS_2019 <- data_by_year_NAICS_2019 %>% arrange(desc(Percent_WFH_jobs))
# create a rank column
data_by_year_NAICS_2019$rank <- 1:nrow(data_by_year_NAICS_2019)
# keep only the naics code and rank columns
data_by_year_NAICS_2019 <- data_by_year_NAICS_2019 %>% ungroup() %>% select(`NAICS 2022 3-Digit Industry Group`,rank)
# merge in into the grouped dataset
data_by_year_NAICS <- merge(data_by_year_NAICS,data_by_year_NAICS_2019,by = "NAICS 2022 3-Digit Industry Group")

In [5]:
names(data_by_year_NAICS)

[1] "NAICS 2022 3-Digit Industry Group"       
[2] "Year"                                    
[3] "NAICS 2022 3-Digit Industry Group (Name)"
[4] "Total_WFH_jobs"                          
[5] "Total_jobs"                              
[6] "Percent_WFH_jobs"                        
[7] "rank"

In [6]:
# keep only year 2019/2023/2024
data_by_year_NAICS <- data_by_year_NAICS %>% filter(Year %in% c(2019,2023,2024))

In [7]:
p <- ggplot(data_by_year_NAICS, aes(x = Percent_WFH_jobs,
 y = reorder(`NAICS 2022 3-Digit Industry Group (Name)`, -1*rank),
 fill = factor(Year))) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8)) +
  labs(
    title = "Percent of job ads offering remote/hybrid vary widely by Industry - Sorted by 2019 Shares",
    x = "Percent",
    y = NULL,
    fill = "Year'"
  ) +
  theme_minimal() +
  scale_fill_manual(values = c("2024" = "#F8766D", "2023" = "#00BFC4","2019" = "#7CAE00")) +
  theme(
    plot.title = element_text(hjust = 0.5, face = "bold"),
    axis.text.y = element_text(size = 10),
    legend.position = "bottom"
  )
ggsave("../output_blogpost_1/figure2.pdf", plot = p, width = 16, height = 32, dpi = 300)

In [8]:
# now make the two subgraphs
# first include on the the top 5 industries
p <- ggplot(data_by_year_NAICS %>% filter(rank <= 5), aes(x = Percent_WFH_jobs,
 y = reorder(`NAICS 2022 3-Digit Industry Group (Name)`, -1*rank),
 fill = factor(Year))) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8)) +
  labs(
    title = "Percent of job ads offering remote/hybrid vary widely by Industry - Top 5 - Sorted by 2019 Shares",
    x = "Percent",
    y = NULL,
    fill = "Year'"
  ) +
  theme_minimal() +
  scale_fill_manual(values = c("2024" = "#F8766D", "2023" = "#00BFC4", "2019" = "#7CAE00")) +
  theme(
    plot.title = element_text(hjust = 0.5, face = "bold"),
    axis.text.y = element_text(size = 10),
    legend.position = "bottom"
  )
ggsave("../output_blogpost_1/figure2_top5.pdf", plot = p, width = 16, height = 10, dpi = 300)

# now make the two subgraphs
# first include on the the bottom 5 industries

p <- ggplot(data_by_year_NAICS %>% filter(rank >= 92), aes(x = Percent_WFH_jobs,
 y = reorder(`NAICS 2022 3-Digit Industry Group (Name)`, -1*rank),
 fill = factor(Year))) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8)) +
  labs(
    title = "Percent of job ads offering remote/hybrid vary widely by Industry - Bottom 5 - Sorted by 2019 Shares",
    x = "Percent",
    y = NULL,
    fill = "Year'"
  ) +
  theme_minimal() +
  scale_fill_manual(values = c("2024" = "#F8766D", "2023" = "#00BFC4", "2019" = "#7CAE00")) +
  theme(
    plot.title = element_text(hjust = 0.5, face = "bold"),
    axis.text.y = element_text(size = 10),
    legend.position = "bottom"
  )
ggsave("../output_blogpost_1/figure2_bottom5.pdf", plot = p, width = 16, height = 10, dpi = 300)
